In [1]:
import threading
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import json
import os
from fake_useragent import UserAgent
from tqdm.notebook import tqdm

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
user_agent = UserAgent()

In [3]:
def get_header():
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
        "Accept-Encoding": "gzip, deflate, br", 
        "Accept-Language": "zh-TW,zh;q=0.9", 
        "Sec-Fetch-Dest": "document", 
        "Sec-Fetch-Mode": "navigate", 
        "Sec-Fetch-Site": "none", 
        "Upgrade-Insecure-Requests": "1", 
        "User-Agent": user_agent.random
    }
    return headers

In [4]:
def get_dist(soup):
    doc = soup.prettify()
    if doc.find('公里') != -1:
        pos = doc.find('公里')
        return float(doc[pos-20:pos].split('\"')[-1])

    elif doc.find('公尺') != -1:
        pos = doc.find('公尺')
        return float(doc[pos-20:pos].split('\"')[-1]) / 1000
    else:
        return None

def job(i, j, url, adjacency_matrix):
    html_doc = requests.get(url, headers=get_header())
    if html_doc.status_code != 200:
        print('I\'m a robot')
        return
    soup = BeautifulSoup(html_doc.text, 'html.parser')
    adjacency_matrix[i['name']][j['name']] = get_dist(soup)

def crawler(addr_list, adjacency_matrix):
    threads = []
    
    arr = []
    for i in addr_list:
        for j in addr_list:
            arr.append([i, j])
            
    for index, (i, j) in enumerate(tqdm(arr)):
        if adjacency_matrix[i['name']][j['name']] != None:
            continue
        url = template.format(i['address'], j['address'])
        print('\r{} -> {}'.format(i['name'], j['name']), end='')
#         print('\r{}/{}, {} -> {}, url: {}'.format(
#             index, len(addr_list)*len(addr_list), i['name'], j['name'], url), end='')

        threads.append(threading.Thread(target = job, args = (i, j, url, adjacency_matrix)))
        threads[-1].start()
    print()
    print('wait a moment')
    for thread in threads:
        thread.join()

def get_valid_edge_size(adjacency_matrix):
    cnt = 0
    for src in adjacency_matrix.keys():
        for dest in adjacency_matrix.keys():
            if adjacency_matrix[src][dest] != None:
                cnt += 1
    return cnt

In [5]:
def init_adjacency_matrix(addr_list):
    adjacency_matrix = {}
    for i in addr_list:
        adjacency_matrix[i['name']] = {}
        for j in addr_list:
            if i['name'] == j['name']:
                adjacency_matrix[i['name']][j['name']] = 0
            else:
                adjacency_matrix[i['name']][j['name']] = None
    return adjacency_matrix

def get_matrix(output_file, addr_list):
    name_set = set(pd.DataFrame(addr_list)['name'])
    
    # cache file
    if os.path.exists(output_file):
        with open(file=output_file, mode='r', encoding='utf-8') as reader:
            adjacency_matrix = json.load(reader)

        # check if the cache file is available
        if set(adjacency_matrix.keys()) == name_set:
            return adjacency_matrix

    adjacency_matrix = init_adjacency_matrix(addr_list)
    # create/overwrite file
    with open(file=output_file, mode='w', encoding='utf-8') as writer:
        json.dump(obj=adjacency_matrix, fp=writer, ensure_ascii=False, indent=4)
    return adjacency_matrix    

In [6]:
template = 'https://www.google.com.tw/maps/dir/{}/{}'
input_file = 'address_list.json'
output_file = 'adjacency_matrix.json'

In [7]:
with open(file=input_file, mode='r', encoding='utf-8') as reader:
    addr_list = json.load(reader)
# addr_list = [{"name": "中坡", "address": "台北市南港區中坡南路47號1樓"},
#               {"name": "中研", "address": "台北市南港區研究院路二段128號1樓(學術活動中心)"},
#               {"name": "中貿", "address": "台北市南港區經貿二路186號2樓"},
#               {"name": "玉成", "address": "台北市南港區西新里南港路三段3號1樓"}]

In [ ]:
adjacency_matrix = get_matrix(output_file, addr_list)

cnt = get_valid_edge_size(adjacency_matrix) # number of edge that already get distance
print('cnt = {}'.format(cnt))

time_out = 0
while cnt != len(addr_list) * len(addr_list):
    if time_out > 3:
        print('[ERROR] I\'m a robot')
        break
    
    crawler(addr_list, adjacency_matrix)
    newcnt = get_valid_edge_size(adjacency_matrix)
    if cnt == newcnt:
        sleep(2)
        time_out += 1

    cnt = newcnt
    print('cnt = {}'.format(cnt))
    sleep(5)

In [9]:
print('adjacency matrix save at {}'.format(output_file))
with open(file=output_file, mode='w', encoding='utf-8') as writer:
    json.dump(obj=adjacency_matrix, fp=writer, ensure_ascii=False, indent=4)

adjacency matrix save at adjacency_matrix.json


In [10]:
pd.DataFrame(adjacency_matrix)

,中坡,中研,中貿,玉成,玉德,向揚,庄研,佳樂,忠陽,昆陽,林坊,研究,胡適,重陽,香城,凱松,港泰,港高鐵,港捷,港勝,港運,港德,港興,港環球,港麗,華技,雄強,慈愛,新福玉,經貿,聯成,聯坊,馥樺,鵬馳,耀港,鑫貿
中坡,0.00,6.00,4.80,2.50,0.35,2.90,5.8,3.40,1.90,1.80,0.65,5.10,5.30,2.80,4.30,1.50,3.40,3.300,4.70,4.30,1.00,1.00,3.80,3.300,2.8,6.4,1.20,3.00,0.50,4.20,1.90,0.65,4.50,2.00,4.10,4.60
中研,5.50,0.00,2.50,4.40,5.30,4.80,1.5,2.80,4.20,4.00,4.90,1.20,0.65,3.90,2.20,5.50,2.80,2.700,2.40,2.00,5.30,5.40,3.10,2.600,3.0,1.9,4.40,3.70,5.80,2.40,3.90,4.80,2.60,4.60,1.90,2.40
中貿,4.70,3.20,0.00,2.70,4.50,3.00,2.9,1.30,3.20,3.20,4.10,2.30,2.50,1.90,0.75,4.70,1.70,1.600,0.45,0.45,4.30,4.60,1.40,1.500,2.0,3.5,3.60,1.70,5.00,0.65,3.10,4.00,0.24,3.50,1.20,0.13
玉成,2.20,4.80,3.10,0.00,2.00,0.40,4.5,1.60,0.45,0.60,1.90,3.90,4.10,1.00,2.90,1.30,1.60,1.800,3.10,2.60,1.60,2.40,1.60,1.800,1.2,5.2,1.30,1.30,2.30,2.40,0.65,1.80,2.50,0.26,2.90,3.00
玉德,0.35,5.70,4.50,2.30,0.00,2.70,5.5,3.20,1.60,1.60,0.40,4.90,5.00,2.50,4.10,1.00,3.20,3.100,4.40,4.10,0.55,1.20,3.50,3.100,2.5,6.1,1.10,2.80,1.00,4.00,1.40,0.50,4.20,1.80,3.80,4.40
向揚,2.40,4.70,2.80,0.40,2.20,0.00,4.4,1.50,1.10,1.00,1.80,3.80,4.00,0.55,2.80,1.70,1.50,2.000,3.00,2.80,2.00,2.30,1.40,2.000,1.3,5.0,1.30,0.80,2.70,2.30,1.10,1.70,2.50,1.10,2.80,3.20
庄研,5.60,1.30,2.60,4.40,5.30,4.80,0.0,2.90,4.30,4.00,5.00,1.10,0.75,4.00,2.30,5.60,2.80,2.700,2.50,2.10,5.40,5.50,3.10,2.700,3.0,0.6,4.40,3.80,5.90,2.40,4.00,4.90,2.70,4.70,2.00,2.40
佳樂,3.40,3.10,1.50,1.60,3.10,2.30,2.9,0.00,1.80,1.80,2.80,2.30,2.40,1.30,0.90,2.90,0.30,0.220,1.40,1.00,2.90,3.30,0.40,0.210,0.6,3.5,2.20,1.10,3.70,1.00,1.80,2.70,1.10,1.90,1.20,1.30
忠陽,1.70,4.30,3.10,0.45,1.50,1.10,4.1,1.70,0.00,0.17,1.20,3.50,3.60,1.10,2.60,1.60,1.80,1.700,3.00,2.60,1.40,1.70,2.10,1.600,1.1,4.7,0.85,1.30,2.10,2.60,0.22,1.10,2.80,0.70,2.40,3.00
昆陽,1.80,4.50,3.30,0.60,1.50,1.30,4.2,1.80,0.17,0.00,1.20,3.60,3.80,1.20,2.70,1.80,1.90,1.800,3.20,2.80,1.50,1.70,2.00,1.800,1.2,4.9,0.80,1.50,2.10,2.60,0.21,1.10,2.90,1.10,2.60,3.10
